In [1]:
import numpy as np
from implementations import *
from cross_validation import *
from data_preprocessing import *
from proj1_helpers import *
import math

print('loading data'+"\n")
DATA_TEST_PATH = '../data/train.csv'
y,tX,ids = load_csv_data(DATA_TEST_PATH)
print('data loaded')

jet_set = jet(tX)
inds = create_inds(jet_set, False)

data_sets = jet_split(tX,inds)
y_sets = split_y(y,inds)


loading data

data loaded


In [4]:
for i in range(len(data_sets)):
    data_sets[i] = replace_aberrant_values(data_sets[i])
    data_sets[i] = standardize(data_sets[i])


In [8]:
matrix = [[1, 1, 1, 1, 1]]
weights_ = []
losses = []
lambda_= np.logspace(1, -8, num = 10)
degree = np.linspace(1,10, num = 9, dtype = int)

for lam in lambda_:
    for deg in degree:       
        for data_set, y_set in zip(data_sets, y_sets):
            test_error, train_error = cross_validation_ridge(y_set, data_set, lam, deg, 0.8)
            losses.append(test_error)
            #losses.append(train_error)
        matrix = np.append(matrix, [[lam, deg, losses[0], losses[1], losses[2]]], axis = 0)
        losses = []

In [6]:
weights=[]
test_error, train_error, w1 = cross_validation_for_leastsquares(y_sets[0], data_sets[0],0.8)
weights.append(w1)
test_error, train_error, w2 = cross_validation_for_leastsquares(y_sets[1], data_sets[1],0.8)
weights.append(w2)
test_error, train_error, w3 = cross_validation_for_leastsquares(y_sets[2], data_sets[2],0.8)
weights.append(w3)


test error = 0.392296076148
train error = 0.39200767659
test error = 0.414327763742
train error = 0.413139532295
test error = 0.369247095985
train error = 0.366028873276


In [8]:

DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
y_preds = predict_merge(tX_test,weights)
OUTPUT_PATH = '../data/submission_splitt.csv'
create_csv_submission(ids_test, y_preds, OUTPUT_PATH)

In [13]:
matrix[20]

array([0.1       , 2.        , 0.42176892, 0.47214813, 0.49712175])

In [14]:
matrix[56]

array([1.00000000e-05, 2.00000000e+00, 4.26361599e-01, 4.74731921e-01,
       4.95908475e-01])